# Join Cleaned Session and Interval Datasets

Based on Oskar's script, but only implement the first part

Idea: having cleaned sessions and intervals data separately, make sure now that there are no sessions which appear in one set but not the other

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
# import ast
import boto3
import s3fs
from io import StringIO

In [3]:
date_saved = '26072019'

In [4]:
interval = pd.read_csv('s3://script.chargepoint.data/Residential_Data/clean/Charging_Intervals_Cleaned'+date_saved+'.csv')
session = pd.read_csv('s3://script.chargepoint.data/Residential_Data/clean/Charging_Sessions_Cleaned'+date_saved+'.csv')

In [5]:
print(list(interval))
print(list(session))

['Interval ID', 'Session ID', 'Interval Start Time (Local)', 'Interval Duration (Secs)', 'Interval Energy', 'Peak Power', 'Average Power']
['Session ID', 'Station ID', 'Port ID', 'Connector Type', 'POI Category', 'POI Subcategory', 'Station Start Time (Local)', 'Station End Time (Local)', 'Session Time Zone', 'Session Time (secs)', 'Charging Time (secs)', 'Start SoC', 'End SoC', 'Full Charge', 'Energy (kWh)', 'Driver ID', 'Driver Zip', 'Vehicle Make', 'Vehicle Model', 'Battery Capacity', 'EV Type', '$ Fee', 'Fee Currency', 'Zip Code', 'City', 'County', 'State', 'Country', 'Driver City', 'Driver State', 'Driver Country', 'Max Power', 'Session Type', 'Vehicle Model Year', 'start_datetime', 'start_seconds', 'start_year', 'start_month', 'start_day', 'start_weekday']


### Helper Fun

In [6]:
def apply_inplace(df, field, fun):
    return pd.concat([df.drop(field, axis=1), df[field].apply(fun)], axis=1) 

In [7]:
def str_list(x):
    return str(list(x))

# Remove Sessions and Intervals without a pair

In [10]:
def remove_unpaired(session, interval):
    # Getting unique session ID values from interval data
    interval_session_IDs_unique = interval['Session ID'].unique()
    session_IDs_unique = session['Session ID'].unique()
    interval_set = set(interval_session_IDs_unique)
    session_set = set(session_IDs_unique)
    paired = interval_set.intersection(session_set)
    print("Number of unique sessions ID in interval data: ", len(interval_session_IDs_unique))
    print("Number of unique sessions ID in session data: ", len(session_IDs_unique))
    print("Number of unique sessions ID in both: ", len(paired))

    # Getting boolean array to check whether session ID from interval data is in session data
    session_has_pair = session['Session ID'].isin(paired)
    # Removing session IDs that are not part of interval data
    session_with_pair = session[session_has_pair]
    
    # Getting boolean array removing interval data that got removed from session data cleaning
    interval_has_pair = interval['Session ID'].isin(paired)
    # Getting final interval data
    interval_with_pair = interval[interval_has_pair]
    
    print("Change of unique sessions ID in session data: ", 
          len(session_with_pair['Session ID'].unique()) - len(session_IDs_unique))
    print("change of unique sessions ID in interval data: ", 
          len(interval_with_pair['Session ID'].unique()) - len(interval_session_IDs_unique))
    
    return session_with_pair, interval_with_pair

In [11]:
session, interval = remove_unpaired(session, interval)

Number of unique sessions ID in interval data:  1022594
Number of unique sessions ID in session data:  1022594
Number of unique sessions ID in both:  1022594
Change of unique sessions ID in session data:  0
change of unique sessions ID in interval data:  0


# Save

In [13]:
# save_date = '26072019'
# Save without date since this will always be the file to use (overwrite each time)

In [ ]:
print("Save paired sessions")
csv_buffer = StringIO()
session.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object('script.chargepoint.data', 'Residential_Data/clean/sessions_paired.csv').put(Body=csv_buffer.getvalue())


print("Save paired intervals")
csv_buffer = StringIO()
interval.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object('script.chargepoint.data', 'Residential_Data/clean/intervals_paired.csv').put(Body=csv_buffer.getvalue())


Save paired sessions
Save paired intervals
